In [ ]:
import pandas as pd
import re
import os
import glob

In [ ]:
os.chdir("DS-Data crawling/data")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [2]:
# Read data from csv file
dataframe = pd.read_csv('data/combined_csv.csv')

In [4]:
dataframe.head()
dataframe = dataframe.iloc[: , 1:]
dataframe
dataframe['Direction'] = dataframe['Direction'].fillna('None')

In [5]:
dataframe.drop(['Address'], axis=1)

,Price,Area,Facade,Building Line,Floors,Bedrooms,Publish Time,Type,ID,Toilets,Direction,District
0,6300,45.0,4.0,10.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468631.269085,None,NaN,Nam Từ Liêm
1,19800,100.0,5.0,30.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468633.5936205,4,NaN,Gia Lâm
2,9800,27.0,5.0,20.0,3,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468636.2328222,2,Tây nam,Tây Hồ
3,12000,95.0,5.0,4.0,5,4,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468641.770186,3,NaN,Long Biên
4,4200,112.0,11.0,2.0,None,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468644.3474865,2,Nam,Nam Từ Liêm
...,...,...,...,...,...,...,...,...,...,...,...,...
2227,24500,224.0,15.0,13.0,4,8,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440674.5257807,5,NaN,Hoàng Mai
2228,7890,48.0,6.0,5.0,6,4,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440679.7092428,5,NaN,Cầu Giấy
2229,2800,40.0,7.0,3.0,4,4,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440682.1170754,3,NaN,Hoài Đức
2230,11000,55.0,4.0,4.0,4,3,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440684.117509,4,NaN,Đống Đa


In [27]:
# Change price unit into billion
val_price = dataframe['price'].values.tolist()
val_area = dataframe['Diện tích'].values.tolist()
for i in range(len(val_area)):
    if isinstance(val_area[i], str):
        val_area[i] = float(val_area[i].replace('m2', ''))
for i in range(len(val_price)):
    if isinstance(val_price[i], str):
        if 'tỷ VNĐ/m2' in val_price[i]:
                val_price[i] = int(float(val_price[i].replace('tỷ VNĐ/m2', '').replace(',', '.'))*val_area[i]* 1e3)
        elif 'tỷ VNĐ' in val_price[i]:
                val_price[i] = int(float(val_price[i].replace('tỷ VNĐ', '').replace(',', '.'))* 1e3)
        elif 'triệu VNĐ/m2' in val_price[i]:
                val_price[i] = int(float(val_price[i].replace('triệu VNĐ/m2', '').replace(',', '.'))*val_area[i])
        elif 'triệu VNĐ' in val_price[i]:
                val_price[i] = int(float(val_price[i].replace('triệu VNĐ', '').replace(',', '.')))
dataframe['price']=val_price
dataframe['Diện tích']=val_area

In [28]:
# Drop nan in numerical attributes
dataframe = dataframe[dataframe['price'].notna()]
dataframe = dataframe[dataframe['Diện tích'].notna()]
dataframe = dataframe[dataframe['Mặt tiền'].notna()]
dataframe = dataframe[dataframe['Lộ giới'].notna()]

In [29]:
dataframe.tail()

,price,address,Diện tích,Mặt tiền,Lộ giới,Số tầng,Số phòng ngủ,Ngày đăng,Loại tin,id,Số toilet,Hướng
11778,24500,"Đường Linh Đàm, Đại Kim, Hoàng Mai",224.0,15.0,13.0,4.0,8.0,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440674.5257807,5.0,NaN
11780,7890,"Đường Nguyễn Khang, Yên Hòa, Cầu Giấy",48.0,6.0,5.0,6.0,4.0,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440679.7092428,5.0,NaN
11781,2800,"Đường Tỉnh Lộ 422B, Vân Canh, Hoài Đức",40.0,7.0,3.0,4.0,4.0,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440682.1170754,3.0,NaN
11782,11000,"Đường Hào Nam, Ô Chợ Dừa, Đống Đa",55.0,4.0,4.0,4.0,3.0,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440684.117509,4.0,NaN
11783,1560,"Đường 1, Ngọc Thụy, Long Biên",62.0,38.0,3.0,2.0,2.0,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440686.7075593,2.0,NaN


In [30]:
dataframe['Số phòng ngủ'] = dataframe['Số phòng ngủ'].fillna('None')
dataframe['Số tầng'] = dataframe['Số tầng'].fillna('None')
dataframe['Số toilet'] = dataframe['Số toilet'].fillna('None')

val_bedrooms = dataframe['Số phòng ngủ'].values.tolist()
val_floors = dataframe['Số tầng'].values.tolist()
val_toilets = dataframe['Số toilet'].values.tolist()
for i in range(len(val_bedrooms)):
    if val_bedrooms[i] != 'None':
        val_bedrooms[i] = str(int(val_bedrooms[i]))
for i in range(len(val_floors)):
    if val_floors[i] != 'None':
        val_floors[i] = str(int(val_floors[i]))
for i in range(len(val_toilets)):
    if val_toilets[i] != 'None':
        val_toilets[i] = str(int(val_toilets[i]))
    
dataframe['Số phòng ngủ']=val_bedrooms
dataframe['Số tầng']=val_floors
dataframe['Số toilet']=val_toilets

In [31]:
# Rename Attributes
dataframe = dataframe.rename(columns={'price': 'Price', 'Diện tích': 'Area', 'Mặt tiền': 'Facade', 'Lộ giới': 'Building Line', 'Số tầng': 'Floors', 'Số phòng ngủ': 'Bedrooms','Ngày đăng': 'Publish Time', 'Số toilet': 'Toilets', 'Hướng': 'Direction', 'Loại tin': 'Type', 'price': 'Price', 'id': 'ID', 'address': 'Address'})

In [32]:
dataframe

,Price,Address,Area,Facade,Building Line,Floors,Bedrooms,Publish Time,Type,ID,Toilets,Direction
0,6300,"Đường Phùng Khoang, Trung Văn, Nam Từ Liêm",45.0,4.0,10.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468631.269085,None,NaN
1,19800,"Đường Trâu Quỳ, Trâu Quỳ, Gia Lâm",100.0,5.0,30.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468633.5936205,4,NaN
2,9800,"Đường Thụy Khuê, Thụy Khuê, Tây Hồ",27.0,5.0,20.0,3,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468636.2328222,2,Tây nam
4,12000,"Đường 1, Bồ Đề, Long Biên",95.0,5.0,4.0,5,4,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468641.770186,3,NaN
5,4200,"Đường Tố Hữu, Trung Văn, Nam Từ Liêm",112.0,11.0,2.0,None,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468644.3474865,2,Nam
...,...,...,...,...,...,...,...,...,...,...,...,...
11778,24500,"Đường Linh Đàm, Đại Kim, Hoàng Mai",224.0,15.0,13.0,4,8,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440674.5257807,5,NaN
11780,7890,"Đường Nguyễn Khang, Yên Hòa, Cầu Giấy",48.0,6.0,5.0,6,4,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440679.7092428,5,NaN
11781,2800,"Đường Tỉnh Lộ 422B, Vân Canh, Hoài Đức",40.0,7.0,3.0,4,4,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440682.1170754,3,NaN
11782,11000,"Đường Hào Nam, Ô Chợ Dừa, Đống Đa",55.0,4.0,4.0,4,3,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440684.117509,4,NaN


In [58]:
# Address Handling
Hanoi_district = ['Hoàn Kiếm', 'Đống Đa', 'Ba Đình', 'Hai Bà Trưng', 'Hoàng Mai', 'Thanh Xuân', 'Long Biên', 'Nam Từ Liêm', 'Bắc Từ Liêm', 'Tây Hồ', 'Cầu Giấy', 'Hà Đông', 'Sơn Tây', 'Ba Vì', 'Chương Mỹ', 'Phúc Thọ', 'Đan Phượng', 'Đông Anh', 'Gia Lâm', 'Hoài Đức', 'Mê Linh', 'Mỹ Đức', 'Phú Xuyên', 'Quốc Oai', 'Sóc Sơn', 'Thạch Thất', 'Thanh Oai', 'Thường Tín', 'Ứng Hòa', 'Thanh Trì']
print(len(Hanoi_district))
dataframe['Address'] = dataframe['Address'].fillna('None')
val_address = dataframe['Address'].values.tolist()
data_district = []
for i in range(len(val_address)):
    if val_address[i] != 'None':
        found = 0
        for j in range(len(Hanoi_district)):
            if Hanoi_district[j].lower() in val_address[i].lower():
                found = 1
                data_district.append(Hanoi_district[j])
                break
        if found == 0:
            data_district.append('None')
    else:
        data_district.append('None')
dataframe['District']=data_district
        

30


In [59]:
dataframe[dataframe['District']=='None']

,Price,Address,Area,Facade,Building Line,Floors,Bedrooms,Publish Time,Type,ID,Toilets,Direction,District
26,3100,ngõ 136 Cầu Diễn,42.0,4.0,3.0,1,1,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468777.4932563,1,NaN,None
99,3800,Lê Quang Đạo,36.0,5.0,5.0,4,3,07/11/2022,Tin thường,batdongsan_so_com_vn_1670469194.539622,3,NaN,None
100,3050,Thụy Phương,55.0,4.0,3.0,3,3,07/11/2022,Tin thường,batdongsan_so_com_vn_1670469197.985291,None,NaN,None
111,14000,Hoàng Quốc Việt,80.0,5.0,40.0,5,6,06/11/2022,Tin thường,batdongsan_so_com_vn_1670469224.986697,8,NaN,None
121,5000,Mễ Trì Thượng,45.0,8.0,3.0,5,3,06/11/2022,Tin thường,batdongsan_so_com_vn_1670469314.2077503,4,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11723,2400,Thụy Phương,45.0,3.0,3.0,2,3,09/11/2022,Tin thường,batdongsan_so_com_vn_1670468355.2416704,None,NaN,None
11725,5300,Phương Canh,100.0,4.0,3.0,1,1,09/11/2022,Tin thường,batdongsan_so_com_vn_1670468412.2266603,None,NaN,None
11741,6000,Văn Tiến Dungx,81.0,8.0,3.0,None,None,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468454.9013958,None,NaN,None
11753,8500,Tân Nhuệ,124.0,8.0,6.0,1,1,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468543.893153,None,NaN,None


In [56]:
dataframe.head()

,Price,Address,Area,Facade,Building Line,Floors,Bedrooms,Publish Time,Type,ID,Toilets,Direction,District
0,6300,"Đường Phùng Khoang, Trung Văn, Nam Từ Liêm",45.0,4.0,10.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468631.269085,None,NaN,Nam Từ Liêm
1,19800,"Đường Trâu Quỳ, Trâu Quỳ, Gia Lâm",100.0,5.0,30.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468633.5936205,4,NaN,Gia Lâm
2,9800,"Đường Thụy Khuê, Thụy Khuê, Tây Hồ",27.0,5.0,20.0,3,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468636.2328222,2,Tây nam,Tây Hồ
4,12000,"Đường 1, Bồ Đề, Long Biên",95.0,5.0,4.0,5,4,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468641.770186,3,NaN,Long Biên
5,4200,"Đường Tố Hữu, Trung Văn, Nam Từ Liêm",112.0,11.0,2.0,None,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468644.3474865,2,Nam,Nam Từ Liêm


In [57]:
(dataframe['District']=='None').sum()

3471

In [61]:
dataframe= dataframe[dataframe['District']!='None']

In [6]:
dataframe.to_csv('data/combined_csv.csv')

In [62]:
dataframe

,Price,Address,Area,Facade,Building Line,Floors,Bedrooms,Publish Time,Type,ID,Toilets,Direction,District
0,6300,"Đường Phùng Khoang, Trung Văn, Nam Từ Liêm",45.0,4.0,10.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468631.269085,None,NaN,Nam Từ Liêm
1,19800,"Đường Trâu Quỳ, Trâu Quỳ, Gia Lâm",100.0,5.0,30.0,4,5,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468633.5936205,4,NaN,Gia Lâm
2,9800,"Đường Thụy Khuê, Thụy Khuê, Tây Hồ",27.0,5.0,20.0,3,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468636.2328222,2,Tây nam,Tây Hồ
4,12000,"Đường 1, Bồ Đề, Long Biên",95.0,5.0,4.0,5,4,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468641.770186,3,NaN,Long Biên
5,4200,"Đường Tố Hữu, Trung Văn, Nam Từ Liêm",112.0,11.0,2.0,None,3,08/11/2022,Tin thường,batdongsan_so_com_vn_1670468644.3474865,2,Nam,Nam Từ Liêm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11778,24500,"Đường Linh Đàm, Đại Kim, Hoàng Mai",224.0,15.0,13.0,4,8,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440674.5257807,5,NaN,Hoàng Mai
11780,7890,"Đường Nguyễn Khang, Yên Hòa, Cầu Giấy",48.0,6.0,5.0,6,4,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440679.7092428,5,NaN,Cầu Giấy
11781,2800,"Đường Tỉnh Lộ 422B, Vân Canh, Hoài Đức",40.0,7.0,3.0,4,4,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440682.1170754,3,NaN,Hoài Đức
11782,11000,"Đường Hào Nam, Ô Chợ Dừa, Đống Đa",55.0,4.0,4.0,4,3,05/12/2022,Tin thường,batdongsan_so_com_vn_1670440684.117509,4,NaN,Đống Đa
